# Storage Manager

In [ ]:
import boto3
import pickle
from io import BytesIO
import os
import fnmatch

class StorageManager:

    def __init__(
        self,
        access_key_id,
        secret_access_key,
        session_token,
        bucket_name,
        bucket_prefix, 
    ):
        self.access_key_id = access_key_id
        self.secret_access_key = secret_access_key
        self.session_token = session_token
        self.bucket_name = bucket_name
        self.bucket_prefix = bucket_prefix
        
        # Init client
        self.s3 = boto3.client(
            's3', 
            aws_access_key_id=access_key_id, 
            aws_secret_access_key=secret_access_key,
            aws_session_token=session_token,
        )
        
    def __fix_path_prefix(self, path_prefix):
        if "/home/ftp" in path_prefix:
            datalake_dir = naas_python.secret.get("ABI_DATALAKE_DIR").value
            path_prefix = path_prefix.replace(f"{datalake_dir}/", "")
        return path_prefix
        
    def pload(
        self,
        path_prefix,
        file_name
    ):
        # Init path
        file_path = os.path.join(self.bucket_prefix, self.__fix_path_prefix(path_prefix), f'{file_name}.pickle')
        try:
            obj = self.s3.get_object(Bucket=self.bucket_name, Key=file_path)
            bytestream = BytesIO(obj['Body'].read())
            return pickle.load(bytestream)
        except Exception as e:
            return None
        return pickle_data

    def pdump(
        self,
        path_prefix,
        object_to_dump,
        file_name,
    ):
        # Init paths
        file_path = os.path.join(self.bucket_prefix, self.__fix_path_prefix(path_prefix), f'{file_name}.pickle')
        histo_path = os.path.join(self.bucket_prefix, self.__fix_path_prefix(path_prefix), f'{datetime.now().strftime("%Y%m%d%H%M%S")}_{file_name}.pickle')
        
        # Save pickle files
        pickle_byte_obj = BytesIO()
        pickle.dump(object_to_dump, pickle_byte_obj)
        pickle_byte_obj.seek(0)
        self.s3.put_object(Bucket=self.bucket_name, Key=file_path, Body=pickle_byte_obj)
        self.s3.put_object(Bucket=self.bucket_name, Key=histo_path, Body=pickle_byte_obj)
        
    def save_data(
        self,
        obj,
        dl_dir,
        entity_name,
        file_name,
    ):
        # Get entity code
        entity_code = unidecode(entity_name.lower().replace(" ", "_").replace(".", ""))

        # Create directory path
        dir_path = os.path.join(dl_dir, entity_code, "tables", file_name)

        # Save in pickle
        self.pdump(dir_path, obj, file_name)
        
    def upload_file(
        self,
        source_path,
        destination_path=None,
    ):
        file_path = self.__fix_path_prefix(source_path)
        if destination_path is not None:
            file_path = self.__fix_path_prefix(destination_path)
        
        self.s3.upload_file(source_path, self.bucket_name, os.path.join(self.bucket_prefix, file_path))
        print(f"✅ Object successfully uploaded: {file_path}")
        return file_path
    
    def list_objects(
        self,
        path_prefix=None,
        path_pattern=None
    ):
        # Init
        files = []
        dir_path = self.bucket_prefix
        pattern = "*"
        if path_prefix is not None:
            dir_path = os.path.join(self.bucket_prefix, self.__fix_path_prefix(path_prefix))
        if path_pattern is not None:
            pattern = "".join(self.__fix_path_prefix(path_pattern))
        
        # List objects
        response = self.s3.list_objects_v2(Bucket=self.bucket_name, Prefix=dir_path + "/")
        
        # Process the response
        if 'Contents' in response:
            for file in response['Contents']:
                file_name = file['Key'].split(self.bucket_prefix + "/")[1]
                if file_name != '' and fnmatch.fnmatch(file_name, pattern):
                    files.append(file_name)
        return files
    
    def delete_object(self, file_name):
        file_path = os.path.join(self.bucket_prefix, self.__fix_path_prefix(file_name))
        self.s3.delete_object(Bucket=self.bucket_name, Key=file_path)
        print("✅ Object successfully deleted!")
        
# sm = StorageManager(access_key_id, secret_access_key, session_token, bucket_name, bucket_prefix)